<a href="https://colab.research.google.com/github/sasamisun/basehtml/blob/master/RinnaTalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
# Huggingface Transformers
!pip install transformers==4.16.0
# Sentencepiece
!pip install sentencepiece==0.1.96

In [ ]:
#@title
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM
import re
 
# トークナイザーとモデルのロード
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")
 
# GPU使用
if torch.cuda.is_available():
    model = model.to("cuda")

In [ ]:
#@markdown ## 名前
my_name = "name" #@param {type:"string"}
#@markdown ## 独り言回数
round_trip = 20 #@param {type:"slider", min:0, max:100, step:1}
 
#@markdown ## 独り言のトピック
topic = "\u5C06\u6765\u3001AI\u306F\u4EBA\u9593\u3088\u308A\u3082\u8CE2\u304F\u306A\u308B\u306E\u304B\u3002" #@param {type:"string"}
text = "りんな:「おはよう" + my_name +"」りんな:「" + topic + "」りんな:「"
 
#@markdown ### parameter変更(Option)
#@markdown 次のトークン確率をモジュール化するために使用される値
temperature = 1 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
#@markdown 繰り返しペナルティのパラメータ。1.0はペナルティなし
repetition_penalty = 0.8 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
#@markdown 長さに対する指数関数的なペナルティ。1.0はペナルティなし
length_penalty = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.1}

In [ ]:
#@title
print("topic:", topic)
 
pos = 3 # 括弧の取得位置
for round_num in range(round_trip):
  token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
  max_length = 100
  if max_length < len(text):
    max_length = len(text) + 30
 
  # りんなちゃんのテキスト生成
  with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_length=max_length,
        min_length=50,
        do_sample=True,
        top_k=500,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_word_ids=[[tokenizer.unk_token_id]],
        temperature = temperature,
        repetition_penalty = repetition_penalty,
        length_penalty = length_penalty
        )
  output = tokenizer.decode(output_ids.tolist()[0])
  # 半角を全角に正規化
  output = output.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
 
  # りんなちゃんの先頭の独り言のみ取得
  prefix = "りんな:「"
  suffix = "」"
  pre = output.split(prefix)
  post = pre[pos].split(suffix)

  # 」で閉じずに言を続けた場合に対処
  if (my_name + ":") in post[0]:
    post[0] = post[0].split(my_name + ":")[0]

  # 」で閉じずにりんなちゃんが次の独り言を続けた場合に対処
  if "りんな:" in post[0]:
    post[0] = post[0].split("りんな:")[0]
  # 」で閉じずに終了した場合
  if "</s>" in post[0]:
    post[0] = post[0].replace("</s>", "")
 
  print(" ->", post[0])
  
  #入力
  ipttxt = input();

  if ipttxt:
    print("")
  else:
    break

  # textに付加
  text += post[0] + my_name + ":「" + ipttxt + "」りんな:「"

 
  # 次回取得位置更新
  pos += 1